# Visualizing the training procces of a Linear model using TensorBoard

### Note:
- Same data set as the one used in [this notebook](Multivariate_linear_regression_PyTorch_Tutorial.ipynb).
- [This](TensorBoard-tutorial.py) is the python script.

# Overview and installation

TensorBoard is part of TensorFlow and offers a framework for visualizing and monitoring the training process. It is useful for visualizing the computational graph of nn models, scalars (loss, classification accuracy), and distributions (weights of the model). 

**Check installation**
```python
>> conda list | grep tensor
>> pip list | grep tensor
```

**Install**
```python
# Using anaconda package manager
>> conda install tensorflow

# Using pip
>> pip install --upgrade pip
>> pip install tensorflow
```

**Check instalation**
```python
>> conda list | grep tensor
```

In [ ]:
import numpy as np
import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter #Class to log data.
from utils import linear_model, SSE

torch.manual_seed(0)
np.random.seed(0)


### Data ###

w_true = torch.tensor(np.array([3.,6.,9.]))       
b_true = torch.tensor([3.])                       
X_true = torch.tensor(np.linspace((0,1,2),(1,2,3),10))
Y_true = linear_model(X_true,w_true,b_true)

Y_obs = torch.add(Y_true, torch.randn(Y_true.shape))


### Model Parameters ###

w_hat = torch.randn(w_true.shape, dtype=torch.float64, requires_grad=True) 
b_hat = torch.randn(1, dtype=torch.float64, requires_grad=True)


### Hyperparamters ### 

alpha  = 0.005      # Learning rate.
n_iter = 10000         # Time steps (epochs).
optimizer = optim.SGD


### TensorBoard Writer Setup ###

# We tell Pytorch where to save a log of the trained weights and loss values.
log_name = f"{optimizer.__name__}_alpha={alpha}"
writer = SummaryWriter(log_dir=f"runs/{log_name}")


### Main Optimization Loop ###

optimizer = optim.SGD([w_hat, b_hat], lr=alpha) 

for t in range(n_iter):               
    optimizer.zero_grad()                                         # Set the gradients to zero.   
    current_loss = SSE(linear_model(X_true, w_hat, b_hat),Y_obs)  # For tracking the loss.
    current_loss.backward()                                       # Compute gradients of loss function (scalar-vector).
    optimizer.step()                                              # Update W_hat and b_hat.
    #if t % 1000 == 0 :
        #print(f"t = {t}, loss = {current_loss}, W_hat = {w_hat.detach().numpy()}, b_hat = {b_hat.item()}")

    # Write the current values of the weights, and loss to the log.
    # global_step=t tells tensorboard at what step of the training this is.
    writer.add_scalar('bias', b_hat, global_step=t)
    writer.add_scalar('w_1', w_hat[0], global_step=t)
    writer.add_scalar('w_2', w_hat[1], global_step=t)
    writer.add_scalar('w_3', w_hat[2], global_step=t)
    writer.add_scalar('L', current_loss, global_step=t)

writer.close() # After we are done with the writer, we should close the log file.

# Starting TensorBoard

**From the terminal**
```python
# Relative path
>> tensorboard --logdir=runs/
>> tensorboard --logdir=runs/ --host localhost --port 8088

# Absolute path
>> tensorboard --logdir=/path/to/directory/
```

**From the notebook**
```python
%load_ext tensorboard
%tensorboard --logdir=/path/to/directory/
```

# Resources

- https://www.tensorflow.org/install
- https://docs.anaconda.com/anaconda/user-guide/tasks/tensorflow/
- https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2017-12-27-optimization.html
- https://pytorch.org/docs/stable/tensorboard.html#:~:text=The%20SummaryWriter%20class%20provides%20a,summaries%20and%20events%20to%20it.&text=Use%20hierarchical%20folder%20structure%20to,experiment%20to%20compare%20across%20them.